# Inverse Kinematics Optimization

The previous doc explained features and how they define objectives of a constrained optimization problem. Here we show how to use this to solve IK optimization problems.

At the bottom there is more general text explaining the basic concepts.

## Demo of features in Inverse Kinematics

Let's setup a standard configuration. (Lock the window with "Always on Top".)

In [1]:
import sys
sys.path += ['../build', '../../../build', '../../lib']
import numpy as np
import libry as ry

**ry-c++-log** ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '
** INFO:ry.cpp:init_LogToPythonConsole:34(0) initializing ry log callback

**ry-c++-log** util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'
** INFO:util.cpp:initCmdLine:545(1) ** cmd line arguments: 'rai-pybind -python '

**ry-c++-log** graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}

** INFO:util.cpp:initCmdLine:549(1) ** run path: '/home/basti/Dokumente/Bachelorarbeit/rai-python/tutorials'

** INFO:graph.cpp:initParameters:1379(1) ** parsed parameters:
{python,
LGP/cameraFocus:[1, 0.5, 1],
LGP/collisions:1,
LGP/stopTime:300,
LGP/stopSol:6,
opt/maxStep:0.1,
opt/verbose:6,
opt/boundedNewton!}



In [2]:
C = ry.Config()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.view()

For simplicity, let's add a frame that represents goals

In [3]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.sphere, [.05])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState() #store the initial configuration

We create an IK engine. The only objective is that the `positionDiff` (position difference in world coordinates) between `pr2L` (the yellow blob in the left hand) and `goal` is equal to zero:

In [15]:
IK = C.komo_IK(False)
IK.addObjective(times=[], type=ry.OT.eq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'])  # missing times=[1.]

We now call the optimizer (True means with random initialization/restart).

In [18]:
IK.optimize()
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:358
** optimization time:0.012375 (kin:0.000771 coll:0 feat:0.004539 newton: 0.001481) setJointStateCount:61
   sos:0.162237 ineq:0 eq:5.48642e-05


0

The best way to retrieve the result is to copy the optimized IK configuration back into your working configuration C, which is now also displayed

In [27]:
C.setFrameState( IK.getFrameState(0) ) # getFrameState() instead of getConfiguration()

We can redo the optimization, but for a different configuration, namely a configuration where the goal is in another location.
For this we move goal in our working configuration C, then copy C back into the IK engine's configurations:

In [36]:
## (iterate executing this cell for different goal locations!)

# move goal
goal.setPosition([.2,.8,.5])

# copy C into the IK's internal configuration(s)
IK.setConfigurations(C)

# reoptimize
IK.optimize(0.) # 0: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getFrameState(0) )

{'F_qItself/1-#46': {'order': 1.0, 'type': 'sos', 'sos': 1.701999998493405e-11}, 'F_qQuaternionNorms/0-#180': {'order': 0.0, 'type': 'eq'}, 'F_PositionDiff/0-pr2L-goal': {'order': 0.0, 'type': 'eq', 'eq': 6.164254625118382e-05}, 'sos': 1.701999998493405e-11, 'ineq': 0.0, 'eq': 6.164254625118382e-05, 'f': 0.0}
** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:360
** optimization time:0.006915 (kin:0.000451 coll:0 feat:0.002034 newton: -289.603) setJointStateCount:1
   sos:1.702e-11 ineq:0 eq:6.16425e-05


Let's solve some other problems, always creating a novel IK engine:

The relative position of `goal` in `pr2R` coordinates equals [0,0,-.2] (which is 20cm straight in front of the yellow blob)

In [39]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(times=[], type=ry.OT.eq, feature=ry.FS.positionRel, frames=['goal','pr2R'], target=[0,0,-.2])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:360
** optimization time:0.003748 (kin:0.000223 coll:0 feat:0.001498 newton: 0.000447) setJointStateCount:63
   sos:0.0138325 ineq:0 eq:0.000200666


The distance between `pr2R` and `pr2L` is zero:

In [41]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(times=[1.], type=ry.OT.eq, feature=ry.FS.distance, frames=['pr2L','pr2R'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:360
** optimization time:0.004508 (kin:0.000287 coll:0 feat:0.001834 newton: 0.000561) setJointStateCount:58
   sos:0.00744107 ineq:0 eq:8.33449e-05


The 3D difference between the z-vector of `pr2R` and the z-vector of `goal`:

In [42]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(times=[1.], type=ry.OT.eq, feature=ry.FS.vectorZDiff, frames=['pr2R', 'goal'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:360
** optimization time:0.004089 (kin:0.000261 coll:0 feat:0.001541 newton: 0.00052) setJointStateCount:65
   sos:0.0208191 ineq:0 eq:1.7085e-06


The scalar product between the z-vector of `pr2R` and the z-vector of `goal` is zero:

In [43]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(times=[1.], type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['pr2R', 'goal'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )

** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:360
** optimization time:0.003587 (kin:0.000243 coll:0 feat:0.001394 newton: 0.000409) setJointStateCount:61
   sos:0.00486204 ineq:0 eq:7.04173e-07


etc etc

## More explanations

All methods to compute paths or configurations solve constrained optimization problems. To use them, you need to learn to define constrained optimization problems. Some definitions:

* An objective defines either a sum-of-square cost term, or an equality constraint, or an inequality constraint in the optimization problem. An objective is defined by its type and its feature. The type can be `sos` (sum-of-squares), `eq`, or `ineq`, referring to the three cases.
* A feature is a (differentiable mapping) from the decision variable (the full path, or robot configuration) to a feature space. If the feature space is, e.g., 3-dimensional, this defines 3 sum-of-squares terms, or 3 inequality, or 3 equality constraints, one for each dimension. For instance, the feature can be the 3-dim robot hand position in the 15th time slice of a path optimization problem. If you put an equality constraint on this feature, then this adds 3 equality constraints to the overall path optimization problem.
* A feature is defined by the keyword for the feature map (e.g., `pos` for position), typically by a set of frame names that tell which objects we refer to (e.g., `pr2L` for the left hand of the pr2), optionally some modifiers (e.g., a scale or target, which linearly transform the feature map), and the set of configuration IDs or time slices the feature is to be computed from (e.g., `confs=[15]` if the feat is to be computed from the 15th time slice in a path optimization problem).
* In path optimization problems, we often want to add objectives for a whole time interval rather than for a single time slice or specific configuration. E.g., avoid collisions from start to end. When adding objectives to the optimization problem we can specify whole intervals, with `times=[1., 2.]`, so that the objective is added to each configuration in this time interval.
* Some features, especially velocity and acceleration, refer to a tuple of (consecutive) configurations. E.g., when you impose an acceleration feature, you need to specify `confs=[13, 14, 15]`. Or if you use `times=[1.,1.]`, the acceleration features is computed from the configuration that corresponds to time=1 and the two configurations *before*.
* All kinematic feature maps (that depend on only one configuration) can be modified to become a velocity or acceleration features. E.g., the position feature map can be modified to become a velocity or acceleration feature.
* The `sos`, `eq`, and `ineq` always refer to the feature map to be *zero*, e.g., constraining all features to be equal to zero with `eq`. This is natural for many features, esp. when they refer to differences (e.g. `posDiff` or `posRel`, which compute the relative position between two frames). However, when one aims to constrain the feature to a non-zero constant value, one can modify the objective with a `target` specification.
* Finally, all features can be rescaled with a `scale` specification. Rescaling changes the costs that arise from `sos` objectives. Rescaling also has significant impact on the convergence behavior for `eq` and `ineq` constraints. As a guide: scale constraints so that if they *would* be treated as squared penalties (squaredPenalty optim mode, to be made accessible) convergence to reasonable approximate solutions is efficient. Then the AugLag will also converge efficiently to precise constraints.

In [ ]:
# Designing a cylinder grasp

In [19]:
D=0
C=0

In [20]:
import sys
sys.path += ['../build', '../../../build', '../../lib']
import numpy as np
import libry as ry

In [21]:
C = ry.Config()
C.addFile('../rai-robotModels/pr2/pr2.g')
C.addFile('../rai-robotModels/objects/kitchen.g')
C.view()

In [22]:
C.setJointState([.7], ["l_gripper_l_finger_joint"])
C.setJointState( C.getJointState() )

In [23]:
goal = C.addFrame("goal")
goal.setShape(ry.ST.cylinder, [0,0,.2, .03])
goal.setColor([.5,1,1])
goal.setPosition([1,.5,1])
X0 = C.getFrameState()

In [24]:
C.setFrameState(X0)
goal.setPosition([1,.5,1.2])
IK = C.komo_IK(False)
IK.addObjective(times=[1.], type=ry.OT.eq,   feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[1,0,0],[0,1,0]])
IK.addObjective(times=[1.], type=ry.OT.ineq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[0,0,1]], target=[0,0,.05])
IK.addObjective(times=[1.], type=ry.OT.ineq, feature=ry.FS.positionDiff, frames=['pr2L', 'goal'], scale=[[0,0,-1]], target=[0,0,-.05])
IK.addObjective(times=[1.], type=ry.OT.sos,  feature=ry.FS.scalarProductZZ, frames=['pr2L', 'goal'], scale=[0.1])
IK.addObjective(times=[1.], type=ry.OT.eq,   feature=ry.FS.scalarProductXZ, frames=['pr2L', 'goal'])
IK.optimize()
C.setFrameState( IK.getFrameState(0) )
IK.getReport()

** KOMO::run solver:dense collisions:0 x-dim:27 T:1 k:1 phases:1 stepsPerPhase:1 tau:1  #timeSlices:2 #totalDOFs:27 #frames:358
** optimization time:0.006323 (kin:0.00034 coll:0 feat:0.002885 newton: 0.000637) setJointStateCount:68
   sos:0.216914 ineq:6.1004e-06 eq:1.51731e-05


{'F_qItself/1-#46': {'order': 1.0, 'type': 'sos', 'sos': 0.21571947990823534},
 'F_qQuaternionNorms/0-#179': {'order': 0.0, 'type': 'eq'},
 'F_PositionDiff/0-pr2L-goal': {'order': 0.0,
  'type': 'ineq',
  'ineq': 6.10040232155995e-06},
 'F_ScalarProduct/0-pr2L-goal': {'order': 0.0,
  'type': 'eq',
  'eq': 1.7029676842583719e-06},
 'sos': 0.21691447831529173,
 'ineq': 6.10040232155995e-06,
 'eq': 1.5173093318665565e-05,
 'f': 0.0}

In [ ]:
IK.view(True, "test")